In [ ]:
import pandas as pd
import json

In [ ]:
transactions = pd.read_csv("transactions_train.csv")
labels = pd.read_json("train_fraud_labels.json")
cards = pd.read_csv("cards_data.csv")
users = pd.read_csv("users_data.csv")
with open("mcc_codes.json", "r") as f:
    mcc_codes = json.load(f)

mcc = pd.DataFrame(
    list(mcc_codes.items()),
    columns=["mcc", "mcc_description"]
)


In [ ]:
transactions.columns

Index(['transaction_id', 'date', 'client_id', 'card_id', 'amount', 'use_chip',
       'merchant_id', 'merchant_city', 'merchant_state', 'zip', 'mcc',
       'errors', 'date_only'],
      dtype='object')

In [ ]:
transactions.head(), labels.head(), cards.head(), users.head(), mcc.head()

In [ ]:
cards.head()

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed,card_on_dark_web
0,4524,825,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No
1,2731,825,Visa,Debit,4956965974959986,12/2020,393,YES,2,$21968,04/2014,2014,No
2,3701,825,Visa,Debit,4582313478255491,02/2024,719,YES,2,$46414,07/2003,2004,No
3,42,825,Visa,Credit,4879494103069057,08/2024,693,NO,1,$12400,01/2003,2012,No
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,03/2009,75,YES,1,$28,09/2008,2009,No


#Labels join

In [ ]:
labels.head()

,target
19765990,No
22160255,No
17566794,No
17318690,No
20994060,No


In [ ]:
labels = labels.reset_index()
labels = labels.rename(columns={"index": "transaction_id"})
labels["target"] = labels["target"].map({"Yes": 1, "No": 0})
labels.head()

,transaction_id,target
0,19765990,0
1,22160255,0
2,17566794,0
3,17318690,0
4,20994060,0


In [ ]:
df = transactions.merge(labels, on="transaction_id", how="left")

#Cards join

In [ ]:
cards.head()

,id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed
0,4524,825,Visa,Debit,4344676511950444,2022-12-01,623,1,2,24295.0,2002-09-01,2008
1,2731,825,Visa,Debit,4956965974959986,2020-12-01,393,1,2,21968.0,2014-04-01,2014
2,3701,825,Visa,Debit,4582313478255491,2024-02-01,719,1,2,46414.0,2003-07-01,2004
3,42,825,Visa,Credit,4879494103069057,2024-08-01,693,0,1,12400.0,2003-01-01,2012
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,2009-03-01,75,1,1,28.0,2008-09-01,2009


In [ ]:
cards = cards.rename(columns={"id": "card_id"})
cards = cards.rename(columns={"client_id": "client_id_2"})

In [ ]:
cards.head()

,card_id,client_id,card_brand,card_type,card_number,expires,cvv,has_chip,num_cards_issued,credit_limit,acct_open_date,year_pin_last_changed
0,4524,825,Visa,Debit,4344676511950444,2022-12-01,623,1,2,24295.0,2002-09-01,2008
1,2731,825,Visa,Debit,4956965974959986,2020-12-01,393,1,2,21968.0,2014-04-01,2014
2,3701,825,Visa,Debit,4582313478255491,2024-02-01,719,1,2,46414.0,2003-07-01,2004
3,42,825,Visa,Credit,4879494103069057,2024-08-01,693,0,1,12400.0,2003-01-01,2012
4,4659,825,Mastercard,Debit (Prepaid),5722874738736011,2009-03-01,75,1,1,28.0,2008-09-01,2009


In [ ]:
df = df.merge(cards, on="card_id", how="left")


#users

In [ ]:
users.head()

,id,current_age,retirement_age,birth_year,birth_month,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,gender_encoded
0,825,53,66,1966,11,462 Rose Lane,34.15,-117.76,29278.0,59696.0,127613.0,787,5,1
1,1746,53,68,1966,12,3606 Federal Boulevard,40.76,-73.74,37891.0,77254.0,191349.0,701,5,1
2,1718,81,67,1938,11,766 Third Drive,34.02,-117.89,22681.0,33483.0,196.0,698,5,1
3,708,63,63,1957,1,3 Madison Street,40.71,-73.99,163145.0,249925.0,202328.0,722,4,1
4,1164,43,70,1976,9,9620 Valley Stream Drive,37.76,-122.44,53797.0,109687.0,183855.0,675,1,0


In [ ]:
users = users.rename(columns={"id": "client_id"})

In [ ]:
users.head()

,client_id,current_age,retirement_age,birth_year,birth_month,address,latitude,longitude,per_capita_income,yearly_income,total_debt,credit_score,num_credit_cards,gender_encoded
0,825,53,66,1966,11,462 Rose Lane,34.15,-117.76,29278.0,59696.0,127613.0,787,5,1
1,1746,53,68,1966,12,3606 Federal Boulevard,40.76,-73.74,37891.0,77254.0,191349.0,701,5,1
2,1718,81,67,1938,11,766 Third Drive,34.02,-117.89,22681.0,33483.0,196.0,698,5,1
3,708,63,63,1957,1,3 Madison Street,40.71,-73.99,163145.0,249925.0,202328.0,722,4,1
4,1164,43,70,1976,9,9620 Valley Stream Drive,37.76,-122.44,53797.0,109687.0,183855.0,675,1,0


In [ ]:
df.columns

Index(['transaction_id', 'date', 'client_id_x', 'card_id', 'amount',
       'use_chip', 'merchant_id', 'merchant_city', 'merchant_state', 'zip',
       'mcc', 'errors', 'date_only', 'target', 'client_id_y', 'card_brand',
       'card_type', 'card_number', 'expires', 'cvv', 'has_chip',
       'num_cards_issued', 'credit_limit', 'acct_open_date',
       'year_pin_last_changed'],
      dtype='object')

In [ ]:
df = df.merge(users, on="client_id", how="left")

In [ ]:
df.columns

Index(['transaction_id', 'date', 'client_id_x', 'card_id', 'amount',
       'use_chip', 'merchant_id', 'merchant_city', 'merchant_state', 'zip',
       'mcc', 'errors', 'date_only', 'target', 'client_id_y', 'card_brand',
       'card_type', 'card_number', 'expires', 'cvv', 'has_chip',
       'num_cards_issued', 'credit_limit', 'acct_open_date',
       'year_pin_last_changed'],
      dtype='object')

#mcc

In [ ]:
mcc.head()

,mcc,mcc_description
0,5812,Eating Places and Restaurants
1,5541,Service Stations
2,7996,"Amusement Parks, Carnivals, Circuses"
3,5411,"Grocery Stores, Supermarkets"
4,4784,Tolls and Bridge Fees


In [ ]:
df.columns

Index(['transaction_id', 'date', 'client_id', 'card_id', 'amount', 'use_chip',
       'merchant_id', 'merchant_city', 'merchant_state', 'zip', 'mcc',
       'errors', 'target', 'current_age', 'retirement_age', 'birth_year',
       'birth_month', 'gender', 'address', 'latitude', 'longitude',
       'per_capita_income', 'yearly_income', 'total_debt', 'credit_score',
       'num_credit_cards'],
      dtype='object')

In [ ]:
df["mcc"] = df["mcc"].astype(str)
mcc["mcc"] = mcc["mcc"].astype(str)
df = df.merge(mcc, on="mcc", how="left")


#df final

In [ ]:
df.columns

Index(['transaction_id', 'date', 'client_id_x', 'card_id', 'amount',
       'use_chip', 'merchant_id', 'merchant_city', 'merchant_state', 'zip',
       'mcc', 'errors', 'date_only', 'target', 'current_age', 'retirement_age',
       'birth_year', 'birth_month', 'address', 'latitude', 'longitude',
       'per_capita_income', 'yearly_income', 'total_debt', 'credit_score',
       'num_credit_cards', 'gender_encoded', 'client_id_y', 'card_brand',
       'card_type', 'card_number', 'expires', 'cvv', 'has_chip',
       'num_cards_issued', 'credit_limit', 'acct_open_date',
       'year_pin_last_changed', 'mcc_description'],
      dtype='object')

In [ ]:
df.to_csv("df-final.csv")